# 上傳檔案 - SigmaDSP

### 需先安裝 ampy ( Adafruit MicroPython Tool )
#### pip install adafruit-ampy
https://github.com/adafruit/ampy  

- In case of "ESP8266/ESP32 - Micropython "Could not enter raw repl": 
    - http://marsdiy.blogspot.com/2018/06/nodemcumircopythoncould-not-enter-raw.html
    - https://github.com/scientifichackers/ampy/issues/19
    - add 'time.sleep(2) as ampy\pyboard.py:[line 187]

[Blog](https://wei1234c.blogspot.tw/2017/08/sx127x-lora-transceiver-driver-for.html)  

In [1]:
import os, sys

def gen_relative_path(target_path): 
    
    def del_same_parents(target_path, current_path):
        if len(target_path) * len(current_path) > 0:
            if target_path[0] == current_path[0]:
                target_path.pop(0)
                current_path.pop(0)
                del_same_parents(target_path, current_path)
            
    current_path = os.getcwd().replace('\\', '/').split('/')
    target_path = target_path.replace('\\', '/').split('/')
    del_same_parents(target_path, current_path)
    
    return ['..'] * len(current_path) + target_path


def append_source_relative_path(source_relative_path):
    sys.path.append(os.sep.join(source_relative_path))

In [2]:
# paths = ['']
# paths = [gen_relative_path(p) for p in paths]
# print(paths)

In [3]:
# paths = [['..', '..', 'codes', 'bridges', 'tools']]

# for path in paths:
#     append_source_relative_path(path)

In [4]:
%pylab inline 

Populating the interactive namespace from numpy and matplotlib


In [5]:
import os
import shutil

### 設定COM port (set current COM port), baud rate

In [6]:
!ls /dev/ttyUSB*

'ls' 不是內部或外部命令、可執行的程式或批次檔。


In [7]:
baud_rate = 115200
# baud_rate = 230400
# baud_rate = 460800
# baud_rate = 921600

com_port = 'COM3'
# com_port = 'COM4'
com_port = 'COM6'
# com_port = 'COM7'
com_port = 'COM11'
# com_port = 'COM12'
# com_port = 'COM13'
# com_port = 'COM15'
# com_port = 'COM16'
# com_port = 'COM17'
# com_port = '/dev/ttyUSB0'
# com_port = '/dev/ttyUSB1'

### Utility functions

In [8]:
def clear_local_folder(folder):
    print('\n[Clearing folder {}]'.format(folder))
    for file in os.listdir(folder):
        os.remove(os.path.join(folder, file))
        
        
def copy_one_file_to_local_folder(folder, file, destination_folder):
    print('Copying {} to {}'.format(file, destination_folder))
    shutil.copy(os.path.join(folder, file), destination_folder)
    
    
def copy_all_files_to_local_folder(folders, destination_folder):
    print('\n[Copying all files to upload folder {}]'.format(destination_folder))
    clear_local_folder(destination_folder)    
    
    for folder in folders: 
        for file in os.listdir(folder):
            if (file.endswith('.py') or file.endswith('.mpy')) and not file.startswith('_'):
                copy_one_file_to_local_folder(folder, file, destination_folder)

In [9]:
def list_files_in_device(com_port):
    files = !ampy --port {com_port} --baud {baud_rate} ls
    return sorted(files)


def cat_file_from_device(com_port, file):
    !ampy --port {com_port} --baud {baud_rate} get {file}
    

def delete_file_in_device(com_port, file):
    print('Deleting {}'.format(file))
    !ampy --port {com_port} --baud {baud_rate} rm {file}
    

def delete_files_in_device(com_port):
    print('\n[Deleting all files in device {}]'.format(com_port))
    for file in list_files_in_device(com_port):
        delete_file_in_device(com_port, file)
        
#     try:
#         !ampy --port {com_port} --baud {baud_rate} rmdir {'/'}
#     except Exception as e:
#         print(e)        
        
        
def copy_one_file_to_device(com_port, folder, file, mpy_only = False):
    if mpy_only:
        if file.endswith('.mpy'):
            print('Copying {}'.format(file))
            !ampy --port {com_port} --baud {baud_rate} put {os.path.join(folder, file)}  
    elif file.endswith('.py'):
        print('Copying {}'.format(file))
        !ampy --port {com_port} --baud {baud_rate} put {os.path.join(folder, file)}   
        

def delet_main_in_device(com_port, main_file_names):
    print('Deleting {}'.format(main_file_names))
    files = list_files_in_device(com_port)

    for file in main_file_names:
        if file in files:
            delete_file_in_device(com_port, file)
            
            
def delete_main_and_files_in_device(com_port, main_file_names = ['main.py', 'main.mpy'], delete_first = True, mpy_only = False):
    print('\n[Copying all files to device {}]'.format(com_port))
    delet_main_in_device(com_port, main_file_names)
    if mpy_only: delete_first = True
    if delete_first: delete_files_in_device(com_port)
        

def copy_all_files_to_device(com_port, folder, main_file_names = ['main.py', 'main.mpy'], delete_first = True, mpy_only = False):    
    for file in os.listdir(folder):
        if (file.endswith('.py') or file.endswith('.mpy')) and not file.startswith('_') and not file in main_file_names:
            copy_one_file_to_device(com_port, folder, file, mpy_only)                    
    
    for file in main_file_names:
        if os.path.isfile(os.path.join(folder, file)):
            copy_one_file_to_device(com_port, folder, file, mpy_only)  
            
            
def do_all_to_device(com_port, folder, main_file_names = ['main.py', 'main.mpy'], delete_first = True, mpy_only = False):
    delete_main_and_files_in_device(com_port, main_file_names, delete_first, mpy_only)
    copy_all_files_to_device(com_port, folder, main_file_names, delete_first, mpy_only)

### Copy *.py to device (all needed fils will be put in the same folder)

In [10]:
paths = ['C:\\Users\\Wei\\Dropbox\\Coding\\notebooks\\專案\\待處理\\SigmaDSP\\bitbucket\\github\\codes\\sigma\\factory']
paths = [gen_relative_path(p) for p in paths]
print(paths)

[['..', '..', 'codes', 'sigma', 'factory']]


In [11]:
paths = [['..', '..', '..', '..', '..', 'Signal Generators', 'bitbucket', 'github', 'codes', 'signal_generators'],
         ['..', '..', '..', '..', '..', 'Utilities', 'bitbucket', 'github', 'codes', 'utilities', 'adapters'],
         ['..', '..', '..', '..', '..', 'Utilities', 'bitbucket', 'github', 'codes', 'utilities'],
         ['..', '..', 'codes', 'sigma'],
         ['..', '..', 'codes', 'sigma', 'factory'], 
         ['..', '..', 'codes', 'sigma', 'mini_xml', 'etree'],
         ['..', '..', 'codes', 'sigma', 'mini_xml'],
         ['..', '..', 'codes', 'sigma', 'sigma_dsp'],
         ['..', '..', 'codes', 'sigma', 'sigma_dsp', 'adau'],
#          ['..', '..', 'codes', 'sigma', 'sigma_dsp', 'adau1401'],
         ['..', '..', 'codes', 'sigma', 'sigma_studio', 'project'],
         ['..', '..', 'codes', 'test', 'uPy'],
         ['..', 'Functional test', 'temp'],]

In [12]:
folders = [os.path.sep.join(path) for path in paths]

upload_folder = os.path.sep.join(['upload', 'py'])

copy_all_files_to_local_folder(folders, upload_folder)


[Copying all files to upload folder upload\py]

[Clearing folder upload\py]
Copying interfaces.py to upload\py
Copying modulators.py to upload\py
Copying tools.py to upload\py
Copying peripherals.py to upload\py
Copying numeric.py to upload\py
Copying register.py to upload\py
Copying shift_register.py to upload\py
Copying factory.py to upload\py
Copying ufactory.py to upload\py
Copying ElementTree.py to upload\py
Copying xmltok2.py to upload\py
Copying dsp_processor.py to upload\py
Copying messages.py to upload\py
Copying adau.py to upload\py
Copying project_xml.py to upload\py
Copying main.py to upload\py
Copying test_adau_upy.py to upload\py
Copying adjustable_gain.py to upload\py
Copying adjustable_volume.py to upload\py
Copying arithmetic_operations.py to upload\py
Copying automatic_eq.py to upload\py
Copying beam_forming.py to upload\py
Copying cell.py to upload\py
Copying chime.py to upload\py
Copying clippers.py to upload\py
Copying combo.py to upload\py
Copying counters.py to 

In [16]:
upload_folder_py = os.path.sep.join(['upload', 'py'])

do_all_to_device(com_port, upload_folder, delete_first = False)

print('\n[All done!]')


[Copying all files to device COM11]
Deleting ['main.py', 'main.mpy']
Copying adau.py
Copying adjustable_gain.py
Copying adjustable_volume.py
Copying arithmetic_operations.py
Copying automatic_eq.py
Copying beam_forming.py
Copying cell.py
Copying chime.py
Copying clippers.py
Copying combo.py
Copying counters.py
Copying crossover.py
Copying cross_fade.py
Copying custom_algorithms.py
Copying dc.py
Copying demultiplexers.py
Copying dsp_functions.py
Copying dsp_processor.py
Copying dynamics_processors.py
Copying dynamic_bass.py
Copying dynamic_mixers.py
Copying effects.py
Copying ElementTree.py
Copying envelope.py
Copying factory.py
Copying fir.py
Copying first_order.py
Copying gpio.py
Copying interfaces.py
Copying level_detectors.py
Copying linear_interpolator.py
Copying linear_sweep.py
Copying logarithmic_sweep.py
Copying logic.py
Copying lookup_tables.py
Copying loudness.py
Copying messages.py
Copying miscellaneous.py
Copying mixers.py
Copying modulators.py
Copying multiplexers.py
Copyi

### 單一檔案上傳 (single file upload, in case needed)

In [ ]:
copy_one_file_to_device(com_port, os.path.sep.join(['..', '..', 'codes', 'sigma', 'sigma_dsp', 'adau1401']), 'adau.py') 

In [13]:
copy_one_file_to_device(com_port, os.path.sep.join(['..', '..', 'codes', 'sigma', 'sigma_studio', 'project']), 'project_xml.py') 

Copying project_xml.py


In [ ]:
copy_one_file_to_device(com_port, os.path.sep.join(['..', '..', '..', '..', '..', 'Utilities', 'bitbucket', 'github', 'codes', 'utilities']), 'numeric.py') 

In [ ]:
copy_one_file_to_device(com_port, os.path.sep.join(['..', '..', 'codes', 'sigma', 'sigma_dsp']), 'messages.py') 

In [15]:
copy_one_file_to_device(com_port, os.path.sep.join(['..', '..', 'codes', 'test', 'uPy']), 'demo.xml') 

### 列出檔案 (list files)

In [ ]:
# list_files_in_device(com_port)

### 檢查檔案內容 (check file content)

In [ ]:
# cat_file_from_device(com_port, 'config_lora.py')

### 連網測試 (network config and test)

In [ ]:
# 連上網路
# import network; nic=network.WLAN(network.STA_IF); nic.active(False);  # disable network
# import network; nic=network.WLAN(network.STA_IF); nic.active(True); nic.connect('SSID','password');nic.ifconfig()
# import network; nic=network.WLAN(network.STA_IF); nic.active(True); nic.connect('Kingnet-70M-$370', '');nic.ifconfig()
# import network; nic=network.WLAN(network.STA_IF); nic.ifconfig()
# import network; nic=network.WLAN(network.STA_IF);nic.ifconfig();nic.config('mac');nic.ifconfig((['mac',])

In [ ]:
# Delete all files
# import u_python;u_python.del_all_files();import os;os.listdir()